In [7]:
import numpy as np
import time
from pylablib.devices import Newport

In [6]:

def gradientAscent(deltaSize, epsilon, range, location): ##Only includes x-axis shifting so far, but startingLocation variable supports up to 4 locations (see GradientTesting.ipynb)
    global powerReading 
    powerReading = float(input("insert power meter reading here"))
    global d0
    stage1.move_to(axis=1, position=location[0]+deltaSize)
    global newPowerReading
    newPowerReading = float(input("insert new power meter reading"))
    d0 = (newPowerReading - powerReading)/deltaSize
   
    print(f"We started at {location}. \n Going {deltaSize} yields a dx of {d0}.")
    global newLocation 

    ##making sure gradient ascent doesn't go out of bounds
    if (location[0]+(d0*epsilon)) > range:
        newLocation0 = range
    elif (location[0]+(d0*epsilon)) < -range:
        newLocation0 = -range
    else:
        newLocation0 = location[0]+(d0*epsilon)

    ##need to account for motor having a greater impact moving "backwards" than forwards:
    if newLocation0 < 0:
        newLocation0 *= 0.6 ##should limit extra unintentional movement. Tune based on practice.

    stage1.move_to(axis=1, position=newLocation0)
    newLocation=stage1.get_position(axis=1)
## def moveBy(startingLocation, epsilon, dx, dy, dz, dw): ##determines next location given the info from gradient ascent
 ##   startingLocation = [startingLocation[0]+(dx*epsilon), startingLocation[1]+(dy*epsilon), startingLocation[2]+(dz*epsilon), startingLocation[3]+(dw*epsilon)]



In [53]:
##stage1 = Newport.Picomotor8742()
stage1.set_position_reference(axis=1)
stage1.get_position(axis=1)
startingLocation = [0,0,0,0]
preCouplePower = float(input("What is the maximum output of the laser prior to the coupling?"))

In [81]:
##using a static epsilon/delta
gradientAscent(deltaSize=dDelta, epsilon=dEpsilon, range=100, location=startingLocation)
startingLocation = [newLocation, 0, 0, 0]
dDelta = 30*(preCouplePower-newPowerReading)
dEpsilon = 20*(preCouplePower-newPowerReading)
time.sleep(1) ##letting things move before running again so as to not freak out the motor


We started at [82, 0, 0, 0]. 
 Going 25 yields a dx of 1.0157894736842106.
0.38


In [ ]:
##using a dynamic epsilon/delta
## 25 is a pretty good start -- broad enough to see decent change w/o losing the couple 
## what we want to do: decrease step size as efficiency gets higher to avoid massive overstep and losing the couple
##                     decrease delta size to a lesser extent to avoid overshooting the peak repeatedly
## how to do it: As power meter/photodetector reading increases, decrease both epsilon and delta by a factor proportional to the efficiency reading
## need: "max" wattage/voltage reading, current wattage/voltage reading BEFORE movement. Then dEpsilon = dEpsilon*a*(max - current), dDelta = dDelta*b*(max - current)
## use same photodetectors measuring efficiency to determine scaling of dEpsilon. Efficiency = post-couple-detector/pre-couple-detector,
##                                                                                Scale Factor = pre-couple-detector - post-couple-detector
## for manual mode, measure output from laser right before fiber with powermeter. Store as variable, then use as the "pre-couple-detector".
